<a href="https://colab.research.google.com/github/Indian1999/szombatmi/blob/main/DBSCANclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, cluster
import scipy
import geopy
import folium

In [4]:
city = "Dubai"
dtf = pd.read_csv("directory.csv")
dtf = dtf[dtf["City"] == city]
dtf = dtf[["City","Street Address", "Longitude", "Latitude"]]
dtf.head()

,City,Street Address,Longitude,Latitude
55,Dubai,"Jumeirah, Near Spinney's",55.23,25.17
56,Dubai,"Reel Cinema, Dubai Mall, Second Floor",55.60,24.78
57,Dubai,"Dubai Airport Rd, Terminal 3",55.35,25.25
58,Dubai,"Safa Street, Jumeirah 1, Jumeirah District",55.31,25.27
59,Dubai,DIFC Building No. 5,55.16,25.10


In [5]:
dtf["Potential"] = np.random.randint(low=3, high = 11, size=len(dtf))
dtf["Staff"] = dtf["Potential"].apply(lambda x: int(np.random.rand()*x) + 1)
dtf["Capacity"] = dtf["Potential"] - dtf["Staff"]
dtf["Cost"] = np.random.choice(["high", "medium", "low"], size=len(dtf), p=[0.4, 0.5, 0.1])
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,Cost
55,Dubai,"Jumeirah, Near Spinney's",55.23,25.17,10,9,1,medium
56,Dubai,"Reel Cinema, Dubai Mall, Second Floor",55.60,24.78,6,5,1,medium
57,Dubai,"Dubai Airport Rd, Terminal 3",55.35,25.25,3,2,1,medium
58,Dubai,"Safa Street, Jumeirah 1, Jumeirah District",55.31,25.27,10,1,9,high
59,Dubai,DIFC Building No. 5,55.16,25.10,8,7,1,high


In [6]:
#Határozzuk meg a város koordinátáit:
locator = geopy.geocoders.Nominatim(user_agent="MyCoder")
location = locator.geocode(city)
print(location)
location = [location.latitude, location.longitude]
print(location)

دبي, الإمارات العربية المتحدة
[25.2653471, 55.2924914]


In [7]:
colors = []
for i in dtf["Cost"]:
  if i == "high":
    colors.append("red")
  if i == "medium":
    colors.append("orange")
  if i == "low":
    colors.append("green")
dtf["Color"] = colors
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,Cost,Color
55,Dubai,"Jumeirah, Near Spinney's",55.23,25.17,10,9,1,medium,orange
56,Dubai,"Reel Cinema, Dubai Mall, Second Floor",55.60,24.78,6,5,1,medium,orange
57,Dubai,"Dubai Airport Rd, Terminal 3",55.35,25.25,3,2,1,medium,orange
58,Dubai,"Safa Street, Jumeirah 1, Jumeirah District",55.31,25.27,10,1,9,high,red
59,Dubai,DIFC Building No. 5,55.16,25.10,8,7,1,high,red


In [8]:
map = folium.Map(location=location, tiles="cartodb positron")
dtf.apply(lambda row: folium.CircleMarker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row["Street Address"],
    color=row["Color"],
    fill = True,
    radius = row["Staff"]).add_to(map), axis=1
)
map

In [9]:
coords = dtf[["Latitude", "Longitude"]]
distortions = []
for i in range(1,11):
  model = cluster.KMeans(n_clusters=i, random_state=0, n_init="auto")
  model.fit(coords)
  distortions.append(model.inertia_)

plt.plot(range(1,11), distortions)
plt.show()

In [10]:
model = cluster.KMeans(n_clusters = 5, n_init="auto")
dtf["Cluster Kmeans"] = model.fit_predict(coords)
centers = model.cluster_centers_
print(centers)
dtf.head()

[[25.07793103 55.15137931]
 [25.23076923 55.36076923]
 [24.78       55.6       ]
 [24.54       55.        ]
 [25.20904762 55.26952381]]


,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,Cost,Color,Cluster Kmeans
55,Dubai,"Jumeirah, Near Spinney's",55.23,25.17,10,9,1,medium,orange,4
56,Dubai,"Reel Cinema, Dubai Mall, Second Floor",55.60,24.78,6,5,1,medium,orange,2
57,Dubai,"Dubai Airport Rd, Terminal 3",55.35,25.25,3,2,1,medium,orange,1
58,Dubai,"Safa Street, Jumeirah 1, Jumeirah District",55.31,25.27,10,1,9,high,red,1
59,Dubai,DIFC Building No. 5,55.16,25.10,8,7,1,high,red,0


In [11]:
fig, ax = plt.subplots()
sns.scatterplot(x="Latitude", y="Longitude", data=dtf,
                palette=sns.color_palette("bright", 5),
                hue = "Cluster Kmeans", legend = "brief", ax=ax
                )
ax.scatter(centers[:,0], centers[:, 1], s = 50, c = "black", marker="x")

In [12]:
dtf = dtf.sort_values(by = ["Cluster Kmeans"])
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,Cost,Color,Cluster Kmeans
93,Dubai,"Ground Floor, Sheikh Zayed Road",55.00,24.92,3,3,0,high,red,0
84,Dubai,Sheik Zayed Rd,55.19,25.11,4,1,3,medium,orange,0
85,Dubai,MBU to provide,55.19,25.03,6,1,5,medium,orange,0
90,Dubai,"Jebel Ali Gardens, Gardens III Tunisia Court",55.12,25.04,7,3,4,high,red,0
131,Dubai,"Block 18, Jumeirah Road, Beside Shape Wellness...",55.16,25.10,9,5,4,high,red,0


In [13]:
#határozzuk meg, hogy mely boltok vannak a klasztereik közepéhez legközelebb
kmeans_centers = []
for k in range(5):
  min_distance = float("inf") #végtelen
  for i in range(len(dtf["Cluster Kmeans"])):
    if dtf.iloc[i, 9] == k:
      distance = np.sqrt((centers[k,0] - dtf.iloc[i,3])**2 + (centers[k,1] - dtf.iloc[i,4])**2)
      if distance < min_distance:
        min_distance = distance
        min_place = i
  kmeans_centers.append(min_place)
print(kmeans_centers)

[6, 36, 55, 56, 59]


In [14]:
kmeans_iscentroid = []
for i in range(len(dtf)):
  if i not in kmeans_centers:
    kmeans_iscentroid.append(0)
  else:
    kmeans_iscentroid.append(1)
dtf["Kmeans Iscentroid"] = kmeans_iscentroid
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,Cost,Color,Cluster Kmeans,Kmeans Iscentroid
93,Dubai,"Ground Floor, Sheikh Zayed Road",55.00,24.92,3,3,0,high,red,0,0
84,Dubai,Sheik Zayed Rd,55.19,25.11,4,1,3,medium,orange,0,0
85,Dubai,MBU to provide,55.19,25.03,6,1,5,medium,orange,0,0
90,Dubai,"Jebel Ali Gardens, Gardens III Tunisia Court",55.12,25.04,7,3,4,high,red,0,0
131,Dubai,"Block 18, Jumeirah Road, Beside Shape Wellness...",55.16,25.10,9,5,4,high,red,0,0


In [ ]:
x,y ="Latitude", "Longitude"
color = "Colors Kmeans"
popup = "Street Address"
marker = centers[:,0]
colors = ["#0000FF", "#FFA500", "#00FF00", "#FF0000", "#800080"]
colors_kmeans = []
for i in dtf["Cluster Kmeans"]:
  colors_kmeans.append(colors[i])
dtf.head()

In [ ]:
map = folium.Map(location = location, tiles = "OpenStreetMap", zoom_start=11)
dtf.apply(lambda row: folium.CircleMarker(
    location=[row[x], row[y]], popup=row[popup], color = row["Colors Kmeans"], fill = True, radius=row["Staff"]).add_to(map), axis = 1
)
map